In [49]:
# directly install the bttai files, will be used later for kaggle submission 
# REFERENCED THIS CODE FOR MODEL https://github.com/deepmancer/resnet-cifar-classification/blob/master/ResNet-18/CIFAR10_Classification.ipynb
!pip install kaggle
!kaggle competitions download bttai-ajl-2025

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 407, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


## Note - About file path

You could use the cell above to print the names of the file directories and get the following directories:

```
/kaggle/input/bttai-ajl-2025/sample_submission.csv
/kaggle/input/bttai-ajl-2025/train.csv
/kaggle/input/bttai-ajl-2025/test.csv
/kaggle/input/bttai-ajl-2025/test/test/e0374ae6c1362ff183cfba28ded5421b.jpg
/kaggle/input/bttai-ajl-2025/test/test/437159c605260bdd079af230566af291.jpg
...
...
/kaggle/input/bttai-ajl-2025/train/train/dermatomyositis/11271bdf2598afdd4260db3125e1f6a5.jpg
/kaggle/input/bttai-ajl-2025/train/train/dermatomyositis/732819951dcf2b53d15ea7b8bb123b71.jpg
/kaggle/input/bttai-ajl-2025/train/train/dermatomyositis/6dcc7a8abb5e1c6e670101f4b6231246.jpg
/kaggle/input/bttai-ajl-2025/train/train/dermatomyositis/e63c3b3f0ab8905e204fe467cc7411f9.jpg
...
...
```



## 1. Import Necessary Libraries

In [50]:
# 1. Import Necessary Libraries
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import keras
from keras import Sequential
from keras import layers
from keras.layers import Conv2D,MaxPool2D,Dense,Flatten,BatchNormalization,Dropout, Input
from keras.optimizers import Adam
from PIL import Image
from tensorflow import data as tf_data
import matplotlib.pyplot as plt
from keras.preprocessing import image

from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms,models,datasets
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

from torch import optim
device = 'cuda' if torch.cuda.is_available() else 'cpu'
from glob import glob
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.optim.lr_scheduler import _LRScheduler
import torch.utils.data as data
import random
import warnings
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

import copy
from collections import namedtuple
import os
import random
import shutil
import time

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)

model.eval()
# or any of these variants
# Explanation:
# - pandas and numpy: for data manipulation
# - sklearn: for splitting data and encoding labels
# - tensorflow.keras: for building and training the neural network
from sklearn.manifold import TSNE
from torch import linalg as LA

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [51]:
print(device)
warnings.filterwarnings('ignore')

cpu


## 2. Load Data

Make sure to verify the file paths if you're running on a different platform.

In [52]:
# 2. Load Data
train_df = pd.read_csv('/kaggle/input/bttai-ajl-2025/train.csv')
test_df = pd.read_csv('/kaggle/input/bttai-ajl-2025/test.csv')

# Add .jpg extension to md5hash column to reference the file_name
train_df['md5hash'] = train_df['md5hash'].astype(str) + '.jpg'
test_df['md5hash'] = test_df['md5hash'].astype(str) + '.jpg'

# Combine label and md5hash to form the correct path
train_df['file_path'] = train_df['label'] + '/' + train_df['md5hash']


# ALSO TRAIN AND TEST DIRECTORIES 
# need to change to train_dir = '/kaggle/input/bttai-ajl-2025/train/train/' for kaggle
train_dir = '/kaggle/input/bttai-ajl-2025/train/train'
test_dir = '/kaggle/input/bttai-ajl-2025/test/test'

In [53]:
# Check the first few rows to understand the structure
train_df.head()

,md5hash,fitzpatrick_scale,fitzpatrick_centaur,label,nine_partition_label,three_partition_label,qc,ddi_scale,file_path
0,fd06d13de341cc75ad679916c5d7e6a6.jpg,4,4,prurigo-nodularis,benign-epidermal,benign,NaN,34,prurigo-nodularis/fd06d13de341cc75ad679916c5d7...
1,a4bb4e5206c4e89a303f470576fc5253.jpg,1,1,basal-cell-carcinoma-morpheiform,malignant-epidermal,malignant,NaN,12,basal-cell-carcinoma-morpheiform/a4bb4e5206c4e...
2,c94ce27e389f96bda998e7c3fa5c4a2e.jpg,5,5,keloid,inflammatory,non-neoplastic,1 Diagnostic,56,keloid/c94ce27e389f96bda998e7c3fa5c4a2e.jpg
3,ebcf2b50dd943c700d4e2b586fcd4425.jpg,3,3,basal-cell-carcinoma,malignant-epidermal,malignant,NaN,34,basal-cell-carcinoma/ebcf2b50dd943c700d4e2b586...
4,c77d6c895f05fea73a8f3704307036c0.jpg,1,1,prurigo-nodularis,benign-epidermal,benign,NaN,12,prurigo-nodularis/c77d6c895f05fea73a8f37043070...


## 3. Data Preprocessing


This section demonstrates basic preprocessing techniques. To enhance data quality and model performance, consider incorporating more advanced preprocessing methods.

For further guidance, feel free to take a look at the [Image Preprocessing tutorial](https://colab.research.google.com/drive/1-ItNcRMbZBE6BCwPT-wD8m3YmHqwHxme?usp=sharing)  available in the 'Resources' section of this Kaggle competition.


In [54]:
# Drop DDI and qc for null values cols 

train_df = train_df.drop('ddi_scale', axis=1)
train_df = train_df.drop('qc', axis=1)
# Display


In [55]:
train_df.head(10)

,md5hash,fitzpatrick_scale,fitzpatrick_centaur,label,nine_partition_label,three_partition_label,file_path
0,fd06d13de341cc75ad679916c5d7e6a6.jpg,4,4,prurigo-nodularis,benign-epidermal,benign,prurigo-nodularis/fd06d13de341cc75ad679916c5d7...
1,a4bb4e5206c4e89a303f470576fc5253.jpg,1,1,basal-cell-carcinoma-morpheiform,malignant-epidermal,malignant,basal-cell-carcinoma-morpheiform/a4bb4e5206c4e...
2,c94ce27e389f96bda998e7c3fa5c4a2e.jpg,5,5,keloid,inflammatory,non-neoplastic,keloid/c94ce27e389f96bda998e7c3fa5c4a2e.jpg
3,ebcf2b50dd943c700d4e2b586fcd4425.jpg,3,3,basal-cell-carcinoma,malignant-epidermal,malignant,basal-cell-carcinoma/ebcf2b50dd943c700d4e2b586...
4,c77d6c895f05fea73a8f3704307036c0.jpg,1,1,prurigo-nodularis,benign-epidermal,benign,prurigo-nodularis/c77d6c895f05fea73a8f37043070...
5,9d5a90fa3f6934608add10e698001760.jpg,3,5,prurigo-nodularis,benign-epidermal,benign,prurigo-nodularis/9d5a90fa3f6934608add10e69800...
6,57885e3f5a3c043c3621a06bca196282.jpg,2,1,seborrheic-keratosis,benign-epidermal,benign,seborrheic-keratosis/57885e3f5a3c043c3621a06bc...
7,8adbbbc4e50a0df8b89710dfd495d3c5.jpg,5,4,eczema,inflammatory,non-neoplastic,eczema/8adbbbc4e50a0df8b89710dfd495d3c5.jpg
8,763ed484fcc50bf7b67cc44f95bac95e.jpg,-1,-1,folliculitis,inflammatory,non-neoplastic,folliculitis/763ed484fcc50bf7b67cc44f95bac95e.jpg
9,0198c74d604fde7055671d1b35869664.jpg,3,4,squamous-cell-carcinoma,malignant-epidermal,malignant,squamous-cell-carcinoma/0198c74d604fde7055671d...


In [56]:
# visualize images # TRIGGER WARNING THE PICS ARE KINDA GRAPHIC!!!!
train_ds, val_ds = keras.utils.image_dataset_from_directory(
    "/kaggle/input/bttai-ajl-2025/train/train",             # Path to the directory containing the dataset.
    validation_split=0.2,    # Specify that 20% of the dataset will be used for validation.
    subset="both",           # Create both training and validation datasets from the directory.
    seed=42,                 # Random seed to ensure reproducibility when shuffling the data.
    #image_size=image_size,   # Resize all images to the specified dimensions (180x180).
    batch_size=32,   # Divide the dataset into batches of the specified size (32 images per batch).
)

'''
# commented section bc images I do not want to see them T_T 
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(np.array(images[i]).astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")
'''

Found 2860 files belonging to 21 classes.
Using 2288 files for training.
Using 572 files for validation.


'\n# commented section bc images I do not want to see them T_T \nplt.figure(figsize=(10, 10))\nfor images, labels in train_ds.take(1):\n    for i in range(9):\n        ax = plt.subplot(3, 3, i + 1)\n        plt.imshow(np.array(images[i]).astype("uint8"))\n        plt.title(int(labels[i]))\n        plt.axis("off")\n'

In [57]:
def create_generator(dataframe, directory, batch_size=32, target_size=(128, 128)):
    """
   we didnt modify this function all that much 
    """
    # Fill in the correct flow_from_dataframe parameters
    generator = train_datagen.flow_from_dataframe(
        dataframe=dataframe,
        directory=directory,
        x_col='file_path',  # Use combined path
        y_col='encoded_label',
        target_size=target_size,
        batch_size=batch_size,
        class_mode='raw',
        validate_filenames=False  # Disable strict filename validation
    )
    return generator


# create train data 
# preprocess - from btt ai notebook sample for preprocessing the data for the model 
label_encoder = LabelEncoder()
train_df['encoded_label'] = label_encoder.fit_transform(train_df['label'])

# split the data into training and validation sets
train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=42)

# define image data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)


In [58]:
# encode data 
encoder1 = LabelEncoder()
train_df['encoded_label'] = label_encoder.fit_transform(train_df['label'])
train_data, val_data = train_test_split(train_df, test_size=0.2, random_state = 42)

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)


# Create generators
train_generator = create_generator(train_data, train_dir)
val_generator = create_generator(val_data, train_dir)

Found 2288 non-validated image filenames.
Found 572 non-validated image filenames.


In [59]:
train_data = train_dir
train_data.__len__()
test_data = test_dir
test_data.__len__()

38

In [60]:
'''
# use pytorch to finish the preprocessing 
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os

class CustomDataset(Dataset):
    def __init__(self, dataframe, directory, transform=None):
        self.dataframe = dataframe
        self.directory = directory
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.directory, self.dataframe.iloc[idx, 0])
        image = Image.open(img_name).convert("RGB")
        label = self.dataframe.iloc[idx, 1]  # Adjust according to your labels

        if self.transform:
            image = self.transform(image)

        return image, label

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Standard ImageNet normalization
])

def create_loader(dataframe, directory, batch_size=32, transform=None):
    dataset = CustomDataset(dataframe, directory, transform)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return loader

# Create data loaders for training and validation
train_loader = create_loader(train_data, train_dir, batch_size=32, transform=transform)
val_loader = create_loader(val_data, train_dir, batch_size=32, transform=transform)
'''

'\n# use pytorch to finish the preprocessing \nfrom torch.utils.data import Dataset, DataLoader\nfrom torchvision import transforms\nfrom PIL import Image\nimport os\n\nclass CustomDataset(Dataset):\n    def __init__(self, dataframe, directory, transform=None):\n        self.dataframe = dataframe\n        self.directory = directory\n        self.transform = transform\n\n    def __len__(self):\n        return len(self.dataframe)\n\n    def __getitem__(self, idx):\n        img_name = os.path.join(self.directory, self.dataframe.iloc[idx, 0])\n        image = Image.open(img_name).convert("RGB")\n        label = self.dataframe.iloc[idx, 1]  # Adjust according to your labels\n\n        if self.transform:\n            image = self.transform(image)\n\n        return image, label\n\ntransform = transforms.Compose([\n    transforms.Resize((128, 128)),\n    transforms.ToTensor(),\n    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Standard ImageNet normalization\n]

In [61]:
def create_loader(dataframe, directory, batch_size=32, transform=None):
    dataset = CustomDataset(dataframe, directory, transform)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return loader

# Create data loaders for training and validation
train_loader = create_loader(train_data, train_dir, batch_size=32, transform=transform)
val_loader = create_loader(val_data, train_dir, batch_size=32, transform=transform)

## 4. Build the model


In [62]:
class ResidualBlock(nn.Module):
    def __init__(self, in_len, out_len, stride=1):
        super(ResidualBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(in_len, out_len, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_len)
        self.conv2 = nn.Conv2d(out_len, out_len, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_len)
        
        layers = []
        if stride != 1 or in_len != out_len:
            layers.append(nn.Conv2d(in_len, out_len, kernel_size=1, stride=stride, bias=False))
            layers.append(nn.BatchNorm2d(out_len))
            self.shortcut = nn.Sequential(*layers)

        else:
            self.shortcut = nn.Sequential()
        

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)
        out = self.bn2(out)
        out = self.conv2(out)
        out += self.shortcut(x)
        out = F.relu(out)
        return out
    
    
class ResNet18(nn.Module):
    def __init__(self):
        super(ResNet18, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
                
        self.layer1 = nn.Sequential(
            ResidualBlock(64, 64, stride=1),
            ResidualBlock(64, 64, stride=1)
        )
        self.layer2 = nn.Sequential(
            ResidualBlock(64, 128, stride=2),
            ResidualBlock(128, 128, stride=1)
        )
        self.layer3 = nn.Sequential(
            ResidualBlock(128, 256, stride=2),
            ResidualBlock(256, 256, stride=1)
        )
        
        self.layer4 = nn.Sequential(
            ResidualBlock(256, 512, stride=2),
            ResidualBlock(512, 512, stride=1)
        )
        
        self.linear = nn.Linear(512, 10)

    def create_block(self, out_len, stride):
        layers = []
        layers.append(ResidualBlock(self.in_len, out_len, stride=stride))
        self.in_len = out_len
        layers.append(ResidualBlock(self.in_len, out_len, stride=1))
        return nn.Sequential(*layers)
    
    def get_feature_space(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        return out
    
    def forward(self, x):
        out = self.get_feature_space(x)
        out = self.linear(out)
        return out

## 5. Train the Model


In [63]:
# TODO: Train your model here. 
#https://www.kaggle.com/code/ggsri123/implementing-resnet18-for-image-classification
def train_model(model, train_loader, val_loader, criterion, optimizer, lr_scheduler, num_epochs=50):
    best_accuracy = 0.0
    val_acc_history = []
    train_acc_history = []
    val_loss_history = []
    train_loss_history = []

    steps = ['train', 'validation']
    dataloaders = {'train': train_loader, 'validation': val_loader}

    print(f"Number of epochs = {num_epochs}")
    print('*********************')

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")

        for step in steps:
            print(f"Step = {step}")
            pre_epoch(step, model)
            
            # Initialize accumulators
            running_loss = 0.0
            correct_number = 0
            total = 0

            # Iterate through batches
            for data, targets in dataloaders[step]:
                data, targets = data.to(device), targets.to(device)
                optimizer.zero_grad()

                # Forward pass
                outputs = model(data)
                batch_loss = criterion(outputs, targets)  # Rename loss to batch_loss
                running_loss += batch_loss.item() * data.size(0)  # Accumulate loss
                
                _, preds = torch.max(outputs, 1)
                correct_number += torch.sum(preds == targets.data)
                total += targets.size(0)

                # Backward pass for training step
                if step == 'train':
                    batch_loss.backward()
                    optimizer.step()

            epoch_loss = running_loss / total
            epoch_accuracy = correct_number.double() / total

            if step == 'validation':
                lr_scheduler.step()

                if epoch_accuracy > best_accuracy:
                    best_accuracy = epoch_accuracy
                    torch.save(model.state_dict(), 'best_model.pth')

                val_acc_history.append(epoch_accuracy)
                val_loss_history.append(epoch_loss)

            if step == 'train':
                train_acc_history.append(epoch_accuracy)
                train_loss_history.append(epoch_loss)

            print(f"{step.capitalize()} Loss: {epoch_loss:.4f} Accuracy: {epoch_accuracy:.4f}")

        print('*********************')

    return train_acc_history, val_acc_history, train_loss_history, val_loss_history


In [64]:
model = ResNet18()
model.to(device)

# this will instantiate the model

ResNet18(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1,

In [65]:
# optimize model 
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.000125)
criterion = nn.CrossEntropyLoss()
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [66]:
from torchvision import transforms
# Define transformations for training
train_transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images to 128x128
    transforms.RandomHorizontalFlip(),  # Randomly flip images horizontally
    transforms.ToTensor(),  # Convert PIL image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet mean/std
])

# Define transformations for validation (no augmentation)
val_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class CustomDataset(Dataset):
    def __init__(self, dataframe, directory, transform=None):
        self.dataframe = dataframe  # Ensure this is a DataFrame
        self.directory = directory
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.directory, self.dataframe.iloc[idx, 0])  # Path
        image = Image.open(img_name).convert("RGB")
        label = self.dataframe.iloc[idx, 1]  # Label
        if self.transform:
            image = self.transform(image)
        return image, label

train_dataset = CustomDataset(dataframe=train_df, directory=train_dir, transform=train_transform)
val_dataset = CustomDataset(dataframe=val_data, directory=train_dir, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [67]:
def pre_epoch(step, model):
    if step == 'train':
        model.train()
    elif step == 'validation':
        model.eval()
    else:
        raise ValueError(f"Unknown step: {step}. Expected 'train' or 'validation'.")
        
torch.cuda.empty_cache()

num_epochs = 50

val_acc_history = []
train_acc_history = []
val_loss_history = []
train_loss_history = []

torch.cuda.empty_cache()
best_accuracy = 0.0
steps = ['train', 'validation']
dataloaders = {
    'validation': val_loader,
    'train': train_loader
}

print('number of epochs = {}'.format(num_epochs))
print('*********************')
for epoch in range(num_epochs):
    print('epoch = {}'.format(epoch))
    for step in steps: 
        print('step = {}'.format(step))
        pre_epoch(step, model)
        loss = 0.0
        correct_number = 0

        for data, targets in dataloaders[step]: 
            data = data.to(device)
            targets = targets.to(device)
            optimizer.zero_grad()
            with torch.set_grad_enabled(step == 'train'):
                outputs = model(data)
                loss = criterion(outputs, targets)
                _, preds = torch.max(outputs, 1)
                if step == 'train': 
                    loss.backward()
                    optimizer.step()
                    
            loss += loss.item() * data.size(0)
            correct_number += torch.sum(preds == targets.data)

        if step == 'validation':
            lr_scheduler.step()
            
        epoch_loss = loss / len(dataloaders[step].dataset)
        epoch_accuracy = correct_number.double() / len(dataloaders[step].dataset)


        if step == 'validation':
            if epoch_accuracy > best_accuracy:
                best_accuracy = epoch_accuracy
                torch.save(model.state_dict(), 'cifar10-resnet18.pth')
            val_acc_history.append(epoch_accuracy)
            val_loss_history.append(epoch_loss)               
            
        if step == 'train':
            train_acc_history.append(epoch_accuracy)
            train_loss_history.append(epoch_loss)
        
        print('loss = {:.4f} accuracy = {:.4f}'.format(epoch_loss, epoch_accuracy))

    print('*********************')

val_acc_history = [x.item() for x in val_acc_history]
train_acc_history = [x.item() for x in train_acc_history]
val_loss_history = [x.item() for x in val_loss_history]
train_loss_history = [x.item() for x in train_loss_history]

number of epochs = 50
*********************
epoch = 0
step = train


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/bttai-ajl-2025/train/train/4ff4109275ef98c7cd754dee47249adc.jpg'

## 6. Make Predictions on Test Data

In [ ]:
# 6. Make Predictions on Test Data
def preprocess_test_data(test_df, directory):
    # TODO: create a generator for the test set here.
    # test_datagen = ImageDataGenerator(rescale=1./255)
    # test_generator = test_datagen.flow_from_dataframe(
    #     .... ## set the test_generator here 
    # )
    # return test_generator
    """
    Template for loading and preprocessing test images with optional transforms.
    Accepts the 'transform' argument to preprocess images.
    """
    test_loader = create_loader(test_df, directory, batch_size=32, transform=transform)
    model.eval()
    predictions = []

    with torch.no_grad():
        for inputs, _ in test_loader:  # Assuming test data has no labels
            inputs = inputs.to(device)
            outputs = model(inputs)
            preds = (outputs.squeeze() > 0.5).cpu().numpy()  # For binary classification (adjust for multi-class)
            predictions.extend(preds)

    return predictions

In [ ]:
# Load test data
test_dir = '/kaggle/input/bttai-ajl-2025/test/test/'
# test_generator = preprocess_test_data(test_df, test_dir)

## 7. Generate Predictions

In [ ]:
# TODO
# Generate predictions based on the trained model
# Then, save the predictions into a CSV file for submission
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)
# Generate predictions for the test set
predictions = preprocess_test_data(test_df, test_dir)

# Save predictions to CSV
test_df['predictions'] = predictions
(test_df.to_csv('predictions.csv', index=False))

In [ ]:

create_download_link(test_df)
